In [1]:
from utils import TspInstance, random_solution, weighted
import numpy as np
import pandas as pd

tsp_a_nodes_random = TspInstance("TSPA.csv", "nodes", random_solution)
tsp_b = TspInstance("TSPB.csv", "edges", random_solution)

# Starting solution

# Greedy
1. Choose arbitrarily starting node and generate random/weighted(best heuristic so far) solution
2. Check each move if solution with it it's better
3. Finish, wehn you check all of them and none of them give us better solution.

Randomly selecting solution
1. Randomly choose if inter/intra route move
2. Randomly look for next solution, remembering already visited solutions

In [2]:
def cost_change_inter(tsp, solution, i, j):
    
    i = i[0][0]
    
    if i == 0:
        i_prev = -1
    else:
        i_prev = i - 1

    if i == len(solution) - 1:
        i_next = 0
    else:
        i_next = i + 1
    
    cost_change = (
        tsp.node_costs[j] - tsp.node_costs[i] 
        - tsp.distance_matrix[solution[i_prev], solution[i]] - tsp.distance_matrix[solution[i], solution[i_next]]
        + tsp.distance_matrix[solution[i_prev], j] + tsp.distance_matrix[j, solution[i_next]]
    )
    return cost_change

def cost_change_intra_nodes(tsp, solution, i, j):
    # i is the index of the node to be replaced
    # j is the node to replace i
    # operations adding 4 new edge and removing 4 old edges
    # is it possible to know, which we don't have to remove?
    
    i = i[0][0]
    j = j[0][0]
    
    if i == 0:
        i_prev = -1
    else:
        i_prev = i - 1

    if j == 0:
        j_prev = -1
    else:
        j_prev = j - 1

    if i == len(solution) - 1:
        i_next = 0
    else:
        i_next = i + 1

    if j == len(solution) - 1:
        j_next = 0
    else:
        j_next = j + 1
    
    cost_change = ( 
        tsp.distance_matrix[solution[i_prev], solution[j]] + tsp.distance_matrix[solution[j], solution[i_next]] +
        tsp.distance_matrix[solution[j_prev], solution[i]] + tsp.distance_matrix[solution[i], solution[j_next]]
        - tsp.distance_matrix[solution[i_prev], solution[i]] - tsp.distance_matrix[solution[i], solution[i_next]]
        - tsp.distance_matrix[solution[j_prev], solution[j]] - tsp.distance_matrix[solution[j], solution[j_next]]
    )
    return cost_change

def cost_change_intra_edges(tsp, solution, i, j):
    # i is the index of the first node in the first edge
    # j is the index of the second node in the second edge
    # i and j are not adjacent!
    
    cost_change = (
        tsp.distance_matrix[solution[i], solution[j]] + tsp.distance_matrix[solution[i+1], solution[j+1]] +
        - tsp.distance_matrix[solution[i], solution[i+1]] - tsp.distance_matrix[solution[j], solution[j+1]]
    )
    return cost_change
    
def intra_route_move(tsp, solution, intra_type):
    
    intra_cost = 0
    visited_solutions = set() 
    
    if intra_type == "nodes":
        max_to_visit = len(solution)*(len(solution)-1)
        while True:
            node_a, node_b = np.random.choice(list(solution), 2, replace=False)
            
            if (node_a, node_b) in visited_solutions:
                continue
            
            visited_solutions.add((node_a, node_b))
            cost = cost_change_intra_nodes(tsp, solution, np.where(solution == node_a), np.where(solution == node_b))
            if cost < intra_cost:
                intra_cost = cost
                
                index_a = np.where(solution == node_a)
                index_b = np.where(solution == node_b)
                
                # if the number is higher than len(solution), set it to zero
                new_index_1 = index_a + np.array(1) if index_a + np.array(1) < len(solution) else 0
                new_index_2 = index_b + np.array(1) if index_b + np.array(1) < len(solution) else 0
                
                buf = solution[new_index_1].item()
                solution[new_index_1] = solution[new_index_2].item()
                solution[new_index_2] = buf
                break
            
            if len(visited_solutions) == max_to_visit:
                print("All possible combinations visited")
                intra_cost = 1
                break
    else:
        # we can draw first edge with any node, but second have to start from not node_a and not node_a+1 or node_a-1
        max_to_visit = len(solution)*(len(solution)-3)
        while True:
            node_a, node_b = np.random.choice(range(len(solution)-1), 2, replace=False)
            
            if (node_a, node_b) in visited_solutions or abs(node_a - node_b) == 1:
                continue
            
            visited_solutions.add((node_a, node_b))
            cost = cost_change_intra_edges(tsp, solution, node_a, node_b)
            if cost < intra_cost:
                intra_cost = cost
                # node_a stays at the same position
                # node_a+1 become node_b
                # node_b become node_a+1
                # node b+1 stays at the same position
                
                new_index_1 = node_a + np.array(1) if node_a + np.array(1) < len(solution) else 0
                new_index_2 = node_b
                
                buf = solution[new_index_1].item()
                solution[new_index_1] = solution[new_index_2].item()
                solution[new_index_2]   = buf
                break
        
            if len(visited_solutions) == max_to_visit:
                print("All possible combinations visited")
                intra_cost = 1
                break
            
    return solution, intra_cost

def inter_route_move(tsp, solution):
    
    inter_cost = 0
    visited_solutions = set()
    # number of all possible combinations of nodes we can swap, 100 to unselect, 100 to select
    max_to_visit = len(solution)**2
    
    while True:
        # Optimization
        # if we visited all possible combinations with a certain node, we can skip that node in our search e.g. random.choice
        
        to_unselect = np.random.choice(list(solution))
        to_select = np.random.choice([i for i in range(tsp.size) if i not in solution])
        
        if (to_unselect, to_select) in visited_solutions:
            continue
        
        visited_solutions.add(tuple(solution))
        cost = cost_change_inter(tsp, solution, np.where(solution == to_unselect), to_select)
        
        if cost < inter_cost:
            inter_cost = cost
            solution[np.where(solution == to_unselect)] = to_select
            break
        
        if len(visited_solutions) == max_to_visit:
            print("All possible combinations visited")
            inter_cost = 1 # cuz, generally it's not possible to get positive cost in this situation, but it indicates that we visited all possible combinations
            break
    
    return solution, inter_cost
        
def greedy(tsp: TspInstance, start_node: int):
    intra_type = tsp.intra_type
    start_sol_method = tsp.start_sol_method
    solution = start_sol_method(tsp, start_node)
    
    counter = 0
    while True:
        counter += 1
    
        if np.random.rand() < 0.5:
            solution, cost = inter_route_move(tsp, solution)
            if solution[1] == 1:
                solution, cost = intra_route_move(tsp, solution, intra_type)
        else:
            solution, cost = intra_route_move(tsp, solution, intra_type)
            if solution[1] == 1:
                solution, cost = inter_route_move(tsp, solution)
                
        if cost == 1:
            break
        
    return solution

In [ ]:
experiments = (
    tsp_b.run_experiments(greedy),
    tsp_a_nodes_random.run_experiments(greedy),
)